<a href="https://colab.research.google.com/github/ali-rn/DS2002F24/blob/main/Nilforoush_Ali_HW_API_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Takes user input for a stock (using the Ticker Symbol): IE with the input() command
2. You will display back the user: Ticker Name, Full Name of the Stock, Current Market
Price, Target Mean Price, 52 week high and 52 week low. Also, output 5 of the current
trending stock.
3. Put that data in a DataFrame and then store that to a CSV file
4. Handle Errors (IE, the stock doesn’t exist and/or the API is not returning information)

In [1]:
import json
import pandas as pd
import json
import requests

# 1. take user input for stock (using the Ticker Symbol):
stock=input("Insert ticker symbol of stock: ")

# 2. display back to user: Ticker Name, Full Name of the Stock, Current Market Price, Target Mean Price, 52 week high & low

# output 5 of the current trending stock

Insert ticker symbol of stock: AAPL
